# IBM Data Science Capstone Project Week 4 + 5

## Explore and find the best location near a subway station in Stuttgart

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a Sushi restaurant. 

Stuttgart is a beatutiful city, but like all the other big cities, it has frequent traffic jams. Fortunately it good subway system to connect all parts of cities. Sushi is very delicious food liked by young people, who also like to traval with public traffic like subway. Therefore in project we will use subway station insdead of neighbourhood to explore the restaurants.

The first criteria for a good location of restaurant should be number of people who is passing by. In case of stuttgart is should be near the city center. The second criteria should be the number of exsitig restaurant, the best location has been choosen thousand times by other people, we can trust it is a good location if there are many restaurant has been opened there.
But we can avoid Sushi restaurant. We don't want to open a Sushi restaurant near a exsiting one. This is then the third criteria.

We will use data analysis and machine learning methodology to explore and find out the possible location, which meet the criteria we defined.

## Data <a name="data"></a>

We need the following data for our problem:
* the location of all subway stations in stuttgart, this can be found in Wikipedia by using Web Scraping of HTML 
* total number of existing restaurants near subway station, it will be obtained using **Foursquare API**
* total number of existing Sushi restaurants near subway station, it can also be find out using **Foursquare API**
* distance of subway from city center, this can be calculated using the infomation of subway station from the Wikipedia




#### Web Scraping of the location data of Stuttgart subway station from Wikipedia

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup 

import requests # library to handle requests

In [5]:
#wikipedia page of subway stations
url = "https://de.wikipedia.org/wiki/Liste_der_Haltestellen_der_Stadtbahn_Stuttgart"
data = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")   

In [6]:

table_contents=[] # define empty date
table=soup.find('table') # find table
table_body = table.find('tbody')

rows = table_body.find_all('tr') 
#get column name from the first row
column = rows[0] #get column
column_name = column.find_all('th')
column_name = [ele.text.strip() for ele in column_name]
# get the content of the table
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    table_contents.append(cols) 

# create pandas dataframe
dataframe= pd.DataFrame(table_contents)
# change the column name
dataframe.columns = column_name
dataframe.head(10)

,Name,Linie,Eröffnung,Lage,Stadt,Stadtteil (Stadtbezirk),Anmerkungen,Sehenswürdigkeiten /\nBauwerke,Bild
0,"Antwerpener Straße48° 48′ 30″ N, 9° 14′ 35″ O",U001 11 U016 1616,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Espan (Bad Cannstatt),Ehemals Friedrich-List-Heim. Erschließt auch d...,,
1,"Arndt-/Spittastraße48° 46′ 30″ N, 9° 9′ 3″ O",U002 22 U009 2929 U014 3434,49890930♠30. Sep. 1989 *,Oberirdisch,Stuttgart,Vogelsang (West),,,
2,"Augsburger Platz48° 48′ 21″ N, 9° 13′ 51″ O",U001 11 U016 1616,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Vorstadt (Bad Cannstatt),Am Augsburger Platz treffen sich die Grenzen d...,"Sankt-Anna-Klinik, Kurpark",
3,"Auwiesen48° 50′ 22″ N, 9° 13′ 30″ O",U012 1212 U014 1414,49860712♠12. Jul. 1986 *,Oberirdisch,Stuttgart,Mühlhausen (Mühlhausen),,"Kirche Sankt Barbara, Burg Hofen (Ruine)",
4,"Bad Cannstatt Wilhelmsplatz48° 48′ 10″ N, 9° 1...",U001 11 U002 22 U019 1919,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Cannstatt-Mitte (Bad Cannstatt),,"Historische Altstadt, Stadtmauer",
5,"Bad Cannstatt Wilhelmsplatz48° 48′ 12″ N, 9° 1...",U013 1313 U016 1616,49970913♠13. Sep. 1997 *,Oberirdisch,Stuttgart,Cannstatt-Mitte (Bad Cannstatt),Diese Haltestelle erhielt als letzte im gesamt...,,
6,"Beethovenstraße48° 46′ 38″ N, 9° 8′ 0″ O",U002 22 U009 2929,49940924♠24. Sep. 1994,Oberirdisch,Stuttgart,Botnang-Ost (Botnang),,,
7,"Bergfriedhof (Merz-Akademie)48° 47′ 21″ N, 9° ...",U004 44 U009 99,49890930♠30. Sep. 1989 *,Oberirdisch,Stuttgart,Ostheim (Ost),Die Hochbahnsteige stehen versetzt zueinander.,"Bergfriedhof, Hauptzollamt",
8,"Bergheimer Hof48° 48′ 15″ N, 9° 6′ 3″ O",U006 66 U016 1616,49920926♠26. Sep. 1992 *,Oberirdisch,Stuttgart,Bergheim (Weilimdorf),,,
9,"Berliner Platz (Hohe Straße)48° 46′ 38″ N, 9° ...",U002 22 U004 44 U014 1414 U014 3434,49860712♠12. Jul. 1986 *,Oberirdisch,Stuttgart,Neue Vorstadt (Mitte),Ehemals Berliner Platz (Fritz-Elsas-Straße).,"Kultur- und Kongresszentrum Liederhalle, Hoppe...",


In [7]:
#choose only station within Stuttgart
index = dataframe[dataframe['Stadt']!='Stuttgart'].index
dataframe.drop(index , inplace=True)
dataframe.head(10)

,Name,Linie,Eröffnung,Lage,Stadt,Stadtteil (Stadtbezirk),Anmerkungen,Sehenswürdigkeiten /\nBauwerke,Bild
0,"Antwerpener Straße48° 48′ 30″ N, 9° 14′ 35″ O",U001 11 U016 1616,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Espan (Bad Cannstatt),Ehemals Friedrich-List-Heim. Erschließt auch d...,,
1,"Arndt-/Spittastraße48° 46′ 30″ N, 9° 9′ 3″ O",U002 22 U009 2929 U014 3434,49890930♠30. Sep. 1989 *,Oberirdisch,Stuttgart,Vogelsang (West),,,
2,"Augsburger Platz48° 48′ 21″ N, 9° 13′ 51″ O",U001 11 U016 1616,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Vorstadt (Bad Cannstatt),Am Augsburger Platz treffen sich die Grenzen d...,"Sankt-Anna-Klinik, Kurpark",
3,"Auwiesen48° 50′ 22″ N, 9° 13′ 30″ O",U012 1212 U014 1414,49860712♠12. Jul. 1986 *,Oberirdisch,Stuttgart,Mühlhausen (Mühlhausen),,"Kirche Sankt Barbara, Burg Hofen (Ruine)",
4,"Bad Cannstatt Wilhelmsplatz48° 48′ 10″ N, 9° 1...",U001 11 U002 22 U019 1919,49860419♠19. Apr. 1986 *,Oberirdisch,Stuttgart,Cannstatt-Mitte (Bad Cannstatt),,"Historische Altstadt, Stadtmauer",
5,"Bad Cannstatt Wilhelmsplatz48° 48′ 12″ N, 9° 1...",U013 1313 U016 1616,49970913♠13. Sep. 1997 *,Oberirdisch,Stuttgart,Cannstatt-Mitte (Bad Cannstatt),Diese Haltestelle erhielt als letzte im gesamt...,,
6,"Beethovenstraße48° 46′ 38″ N, 9° 8′ 0″ O",U002 22 U009 2929,49940924♠24. Sep. 1994,Oberirdisch,Stuttgart,Botnang-Ost (Botnang),,,
7,"Bergfriedhof (Merz-Akademie)48° 47′ 21″ N, 9° ...",U004 44 U009 99,49890930♠30. Sep. 1989 *,Oberirdisch,Stuttgart,Ostheim (Ost),Die Hochbahnsteige stehen versetzt zueinander.,"Bergfriedhof, Hauptzollamt",
8,"Bergheimer Hof48° 48′ 15″ N, 9° 6′ 3″ O",U006 66 U016 1616,49920926♠26. Sep. 1992 *,Oberirdisch,Stuttgart,Bergheim (Weilimdorf),,,
9,"Berliner Platz (Hohe Straße)48° 46′ 38″ N, 9° ...",U002 22 U004 44 U014 1414 U014 3434,49860712♠12. Jul. 1986 *,Oberirdisch,Stuttgart,Neue Vorstadt (Mitte),Ehemals Berliner Platz (Fritz-Elsas-Straße).,"Kultur- und Kongresszentrum Liederhalle, Hoppe...",


In [8]:
#funtion to coversion of coordination
def converseCoordinate(degree,minute,second):
    coordDec = degree + minute/60 + second/3600
    return coordDec

In [9]:
# define the dataframe columns
column_names = ['Subway Station', 'Line', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
subway_station = pd.DataFrame(columns=column_names)

In [10]:
# read and prepare the information of subway station from the row data
for index, rows in dataframe.iterrows():
    #cell['Borough'] = (row.span.text).split('(')[0]
    #cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    stationID = rows['Name'][0: rows['Name'].find('48')]
    coordinate = rows['Name'][rows['Name'].find('48'):]
    coordinate_lat = coordinate.split('N,')[0]
    coordinate_lon = coordinate.split('N,')[1]
    #print(coordinate_lat)
    #print(coordinate_lon)
    minute_lat = int(coordinate_lat[coordinate_lat.find('°')+1:coordinate_lat.find('′')])
    second_lat = int(coordinate_lat[coordinate_lat.find('′')+1:coordinate_lat.find('″')])
    minute_lon = int(coordinate_lon[coordinate_lat.find('°')+1:coordinate_lon.find('′')])
    second_lon = int(coordinate_lon[coordinate_lat.find('′')+1:coordinate_lon.find('″')])
    station_lat = converseCoordinate(48,minute_lat,second_lat)
    station_lon = converseCoordinate(9,minute_lon,second_lon)
    #print(station_lat)
    #print(station_lon)
    lineID = [i.replace('0','') for i in rows['Linie'].split(' ') if i.find('U')!=-1]
    #print(lineID)
    neighborhood = rows['Stadtteil (Stadtbezirk)'][rows['Stadtteil (Stadtbezirk)'].find('(')+1:rows['Stadtteil (Stadtbezirk)'].find(')')]
    #print(neighborhood)

    subway_station = subway_station.append({'Subway Station': stationID,
                                            'Line': lineID,
                                            'Neighborhood': neighborhood,
                                            'Latitude': station_lat,
                                            'Longitude': station_lon}, ignore_index=True)

In [11]:
subway_station.shape

(188, 5)

In [12]:
subway_station.head()

,Subway Station,Line,Neighborhood,Latitude,Longitude
0,Antwerpener Straße,"[U1, U16]",Bad Cannstatt,48.808333,9.243056
1,Arndt-/Spittastraße,"[U2, U9, U14]",West,48.775000,9.150833
2,Augsburger Platz,"[U1, U16]",Bad Cannstatt,48.805833,9.230833
3,Auwiesen,"[U12, U14]",Mühlhausen,48.839444,9.225000
4,Bad Cannstatt Wilhelmsplatz,"[U1, U2, U19]",Bad Cannstatt,48.802778,9.215833


## Methodology <a name="methodology"></a>

In this project we will explore the restaurant near each subway stations with Foursquare API. And visualize the location with Folium map.

we will count the number of total restaurant and the total Sushi retaurants near the subway station. 

The Distance of the subway station to the city center will be calculated and considered in the analysis.

we will use K means to cluster the subway station base on the all information above. The clasification will be visualized the city map.

Base on the classification we can find a group of candidate of subway station which we can open a Sushi restaurant near them.


## Analysis <a name="analysis"></a>

Import the libaries

In [13]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim

<h4>Getting Stuttgart Coordinates from Geopy library</h4>

In [14]:
address = 'Stuttgart, Germany'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stuttgart are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stuttgart are 48.7784485, 9.1800132.


Visualize the location of the Subway Stations

In [15]:
# create map of New York using latitude and longitude values
map_stuttgart = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, station, neighborhood in zip(subway_station['Latitude'], subway_station['Longitude'], subway_station['Subway Station'], subway_station['Neighborhood']):
    label = '{}, {}'.format(station, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stuttgart)  
    
map_stuttgart

In [ ]:
CLIENT_ID = 'UCAD22UHIEXLJT5GFMM1PLTLT1IF0WP1DVVYQU21DQQYJ5FW' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Creating a function to find vanues of all the subway station within 300 meter

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Subway Station', 
                  'Subway Station Latitude', 
                  'Subway Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
stuttgart_venues = getNearbyVenues(names=subway_station['Subway Station'],
                                   latitudes=subway_station['Latitude'],
                                   longitudes=subway_station['Longitude'],
                                  )

Antwerpener Straße
Arndt-/Spittastraße
Augsburger Platz
Auwiesen
Bad Cannstatt Wilhelmsplatz
Bad Cannstatt Wilhelmsplatz
Beethovenstraße
Bergfriedhof (Merz-Akademie)
Bergheimer Hof
Berliner Platz (Hohe Straße)
Berliner Platz (Liederhalle)
Berliner Platz (Liederhalle)
Beskidenstraße
Bihlplatz
Blick
Bopser
Borsigstraße
Botnang
Bottroper Straße (BiL-Schulen)
Börsenplatz (L-Bank)
Brendle (Großmarkt)
Bubenbad (IB-Jugendgästehaus)
Budapester Platz
Cannstatter Wasen
Charlottenplatz (Ebene -1)
Charlottenplatz (Ebene -2)
Daimlerplatz
Degerloch
Degerloch Albstraße
Dobelstraße
Dürrlewang
Ebitzweg
Elbestraße
Eltinger Straße
EnBW-City
Engelboldstraße
Erwin-Schoettle-Platz
(Marienhospital)
Eszet
Eugensplatz (Jugendherberge)
Europaplatz
Fasanenhof (Bonhoefferkirche)
Fasanenhof Schelmenwasen
Fauststraße
Feuerbach Bahnhof
Feuerbach Pfostenwäldle (AWO)
Föhrich
Freiberg
Freibergstraße
Friedrichswahl
Fürfelder Straße
Gaisburg
Geroksruhe
Giebel
Glockenstraße (Mahle)
Gnesener Straße
Hallschlag
Hauptbahnhof 

#### Let's check how many venues were returned for each subway station

In [19]:
stuttgart_venues.groupby('Subway Station').count()

,Subway Station Latitude,Subway Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Subway Station,,,,,,
Antwerpener Straße,4,4,4,4,4,4
Arndt-/Spittastraße,8,8,8,8,8,8
Augsburger Platz,5,5,5,5,5,5
Auwiesen,5,5,5,5,5,5
Bad Cannstatt Wilhelmsplatz,48,48,48,48,48,48
...,...,...,...,...,...,...
Wolfbusch,2,2,2,2,2,2
Zuffenhausen Kelterplatz,10,10,10,10,10,10
Zuffenhausen Rathaus,15,15,15,15,15,15


In [20]:
stuttgart_venues.head()

,Subway Station,Subway Station Latitude,Subway Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antwerpener Straße,48.808333,9.243056,U Antwerpener Straße,48.808411,9.242957,Metro Station
1,Antwerpener Straße,48.808333,9.243056,Melirrytom,48.808361,9.243657,Pastry Shop
2,Antwerpener Straße,48.808333,9.243056,Fleischmann-MV,48.806688,9.243476,Deli / Bodega
3,Antwerpener Straße,48.808333,9.243056,Creative Logo,48.807610,9.240681,Advertising Agency
4,Arndt-/Spittastraße,48.775000,9.150833,Meister Lampe,48.774808,9.149525,Dessert Shop


Let's find all restaurant 

In [21]:
stuttgart_restaurant = stuttgart_venues[stuttgart_venues['Venue Category'].map(lambda x: x.find('Restaurant')!=-1)]

Let's remove all kind of fast food restaurant

In [ ]:
for index, rows in stuttgart_restaurant.iterrows():
    if rows['Venue Category'] in ['Fast Food Restaurant','Comfort Food Restaurant','Doner Restaurant','Kebab Restaurant']:
        stuttgart_restaurant.drop(index, inplace=True)

In [ ]:
stuttgart_restaurant.reset_index(inplace=True)

Let's check how many restaurant are there

In [26]:
stuttgart_restaurant.groupby('Subway Station').count()

,level_0,index,Subway Station Latitude,Subway Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Subway Station,,,,,,,,
Arndt-/Spittastraße,4,4,4,4,4,4,4,4
Augsburger Platz,1,1,1,1,1,1,1,1
Bad Cannstatt Wilhelmsplatz,13,13,13,13,13,13,13,13
Beethovenstraße,1,1,1,1,1,1,1,1
Berliner Platz (Hohe Straße),5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...
Wilhelma,2,2,2,2,2,2,2,2
Zuffenhausen Kelterplatz,2,2,2,2,2,2,2,2
Zuffenhausen Rathaus,4,4,4,4,4,4,4,4


Let's create a new dataframe with the total number of reataurant

In [89]:
station_summary = stuttgart_restaurant.groupby('Subway Station')['Venue Category'].count().reset_index()

In [90]:
station_summary.rename(columns = {'Venue Category':'Total restaurant'}, inplace = True)

In [91]:
station_summary.head()

,Subway Station,Total restaurant
0,Arndt-/Spittastraße,4
1,Augsburger Platz,1
2,Bad Cannstatt Wilhelmsplatz,13
3,Beethovenstraße,1
4,Berliner Platz (Hohe Straße),5


Let' check all Sushi restaurant

In [92]:
sushi_restaurant = stuttgart_restaurant[stuttgart_restaurant['Venue Category']=='Sushi Restaurant']

In [93]:
sushi_restaurant.head()

,level_0,index,Subway Station,Subway Station Latitude,Subway Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,5,23,Bad Cannstatt Wilhelmsplatz,48.802778,9.215833,sushi le,48.804388,9.216886,Sushi Restaurant
12,12,47,Bad Cannstatt Wilhelmsplatz,48.803333,9.215556,sushi le,48.804388,9.216886,Sushi Restaurant
39,39,206,Budapester Platz,48.790556,9.185833,Tokyo,48.790902,9.183083,Sushi Restaurant
45,45,227,Charlottenplatz (Ebene -1),48.775833,9.182778,QQ Sushi Lounge,48.775175,9.183356,Sushi Restaurant
48,48,235,Charlottenplatz (Ebene -1),48.775833,9.182778,wip sushi lounge,48.774476,9.182102,Sushi Restaurant


Let' calculate the total number of Sushi restaurant

In [94]:
sushi_restaurant_count = sushi_restaurant.groupby('Subway Station')['Venue Category'].count().reset_index()

In [95]:
sushi_restaurant_count.rename(columns = {'Venue Category':'Total Sushi restaurant'}, inplace = True)

In [96]:
sushi_restaurant_count.head()

,Subway Station,Total Sushi restaurant
0,Bad Cannstatt Wilhelmsplatz,2
1,Budapester Platz,1
2,Charlottenplatz (Ebene -1),2
3,Charlottenplatz (Ebene -2),2
4,Daimlerplatz,1


Add the Sushi restaurant infomation into the data frame 

In [97]:
station_summary = station_summary.merge(sushi_restaurant_count, how='outer',on='Subway Station')

In [98]:
station_summary = station_summary.fillna(0)

Let find the location of the subway, we will use mean value in case there are many exit of one station

In [99]:
station_location = stuttgart_restaurant.groupby('Subway Station')['Subway Station Latitude','Subway Station Longitude'].mean().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [100]:
station_location.head(10)

,Subway Station,Subway Station Latitude,Subway Station Longitude
0,Arndt-/Spittastraße,48.775000,9.150833
1,Augsburger Platz,48.805833,9.230833
2,Bad Cannstatt Wilhelmsplatz,48.803034,9.215705
3,Beethovenstraße,48.777222,9.133333
4,Berliner Platz (Hohe Straße),48.777222,9.168611
5,Berliner Platz (Liederhalle),48.778611,9.168117
6,Bihlplatz,48.759722,9.151389
7,Bubenbad (IB-Jugendgästehaus),48.773333,9.194722
8,Budapester Platz,48.790556,9.185833
9,Börsenplatz (L-Bank),48.779722,9.178333


We will define function to caloulate the distance between subway station and city center 

In [39]:
#!pip install pyproj
import pyproj
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

Let's calculate the distance between each subway station and city center 

In [51]:
distance_center = pd.DataFrame(columns= ['Distance'])

In [52]:
for index, rows in station_location.iterrows():
    x1, y1 = lonlat_to_xy(rows['Subway Station Latitude'], rows['Subway Station Longitude'])
    x2, y2 = lonlat_to_xy(latitude,longitude)
    distance = str(calc_xy_distance(x1,y1,x2,y2))
    print(distance)
    
    distance_center = distance_center.append({'Distance': distance}, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


3891.2501100861095


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


7635.814913017591


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


inf
inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


In [87]:
distance_center.head()

,Distance
0,3891.2501100861095
1,7635.814913017591
2,5729.074698031653
3,6184.7048814929085
4,1518.7378735569764


In [101]:
station_summary.insert(3,column='Distance',value=distance_center)

In [102]:
station_summary.head(20)

,Subway Station,Total restaurant,Total Sushi restaurant,Distance
0,Arndt-/Spittastraße,4,0.0,3891.2501100861095
1,Augsburger Platz,1,0.0,7635.814913017591
2,Bad Cannstatt Wilhelmsplatz,13,2.0,5729.074698031653
3,Beethovenstraße,1,0.0,6184.7048814929085
4,Berliner Platz (Hohe Straße),5,0.0,1518.7378735569764
5,Berliner Platz (Liederhalle),9,0.0,1575.7012264169061
6,Bihlplatz,1,0.0,4521.414423314856
7,Bubenbad (IB-Jugendgästehaus),1,0.0,2061.073677377733
8,Budapester Platz,2,1.0,1770.1359677851751
9,Börsenplatz (L-Bank),4,0.0,278.5719176074256


In [103]:
station_summary.shape

(96, 4)

Let normalize the summary of the restaurant and subway station distance infomation 

In [104]:
import pandas as pd
from sklearn import preprocessing
station_summary_norm = station_summary.drop('Subway Station',1)
x = station_summary_norm.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
station_summary_norm = pd.DataFrame(x_scaled)

In [105]:
station_summary_norm.head()

,0,1,2
0,0.15,0.0,0.334357
1,0.00,0.0,0.669888
2,0.60,1.0,0.499035
3,0.00,0.0,0.539861
4,0.20,0.0,0.121768


#### Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 6 clusters.

In [106]:
# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(station_summary_norm)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 1, 3, 0, 0, 5, 0, 4, 0], dtype=int32)

Let's merge the restaurant and location infomation together

In [107]:
station_summary.insert(1, 'Cluster Labels', kmeans.labels_)

stuttgart_merged = station_summary

stuttgart_merged = stuttgart_merged.merge(station_location, on='Subway Station')

stuttgart_merged = stuttgart_merged.reset_index(drop=True)

stuttgart_merged

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Cluster Labels,Subway Station Latitude,Subway Station Longitude
0,Arndt-/Spittastraße,4,0.0,3891.2501100861095,0,48.775000,9.150833
1,Augsburger Platz,1,0.0,7635.814913017591,3,48.805833,9.230833
2,Bad Cannstatt Wilhelmsplatz,13,2.0,5729.074698031653,1,48.803034,9.215705
3,Beethovenstraße,1,0.0,6184.7048814929085,3,48.777222,9.133333
4,Berliner Platz (Hohe Straße),5,0.0,1518.7378735569764,0,48.777222,9.168611
...,...,...,...,...,...,...,...
91,Wilhelma,2,0.0,5000.950811129121,5,48.803889,9.208056
92,Zuffenhausen Kelterplatz,2,0.0,6769.738400755982,3,48.829722,9.176111
93,Zuffenhausen Rathaus,4,0.0,6931.840353161858,3,48.830833,9.174722
94,Züricher Straße,1,0.0,4936.429753320783,5,48.810278,9.199722


Finally, let's visualize the resulting clusters

In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stuttgart_merged['Subway Station Latitude'], stuttgart_merged['Subway Station Longitude'], stuttgart_merged['Subway Station'], stuttgart_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
Now, we can explore each cluster. 

In [118]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 0, stuttgart_merged.columns[[0,1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
0,Arndt-/Spittastraße,4,0.0,3891.2501100861095,48.775000,9.150833
4,Berliner Platz (Hohe Straße),5,0.0,1518.7378735569764,48.777222,9.168611
5,Berliner Platz (Liederhalle),9,0.0,1575.7012264169061,48.778611,9.168117
7,Bubenbad (IB-Jugendgästehaus),1,0.0,2061.073677377733,48.773333,9.194722
9,Börsenplatz (L-Bank),4,0.0,278.5719176074256,48.779722,9.178333
13,Degerloch,5,0.0,4159.278159173068,48.748889,9.168889
16,Erwin-Schoettle-Platz\n(Marienhospital),3,0.0,3358.229092155877,48.762778,9.160000
17,Eugensplatz (Jugendherberge),4,0.0,1397.2177319878526,48.778056,9.190556
24,Hauptbahnhof Arnulf-Klett-Platz,2,0.0,680.3555359069533,48.783611,9.180278
29,Hölderlinplatz,4,0.0,2902.529315539401,48.781667,9.158333


In [121]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 1, stuttgart_merged.columns[[0,1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
2,Bad Cannstatt Wilhelmsplatz,13,2.0,5729.074698031653,48.803034,9.215705
10,Charlottenplatz (Ebene -1),15,2.0,502.50657553364147,48.775833,9.182778
11,Charlottenplatz (Ebene -2),10,2.0,466.2050179406534,48.776667,9.183056
58,Rathaus,21,2.0,600.061270969988,48.773889,9.180000


In [122]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 2, stuttgart_merged.columns[[0,1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
14,Dürrlewang,1,0.0,11319.90646658139,48.719167,9.118056
18,Europaplatz,1,0.0,9096.233444334497,48.712222,9.160278
25,Hedelfingen,1,0.0,10115.279810353206,48.760000,9.254167
26,Heumaden,1,0.0,8895.472911339675,48.738056,9.233889
28,Hofen,1,0.0,9457.525977991196,48.836389,9.222222
34,Landauer Straße,2,0.0,8836.578287163371,48.814722,9.123889
39,Max-Eyth-See,1,0.0,8623.901769347725,48.832778,9.216389
46,Möhringen Freibad,1,0.0,9572.59638172483,48.716944,9.141389
53,Plieningen (Universität Hohenheim),1,0.0,9414.90356464347,48.710278,9.201667
72,Schlotterbeckstraße,1,0.0,9153.079503901692,48.784167,9.248889


In [123]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 3, stuttgart_merged.columns[[0,1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
1,Augsburger Platz,1,0.0,7635.814913017591,48.805833,9.230833
3,Beethovenstraße,1,0.0,6184.7048814929085,48.777222,9.133333
15,Elbestraße,1,0.0,7569.0673037925935,48.825556,9.212778
20,Freibergstraße,1,0.0,8189.683792483637,48.824167,9.221944
21,Fürfelder Straße,1,0.0,7253.292880765147,48.832222,9.191944
23,Hallschlag,1,0.0,6167.453734544554,48.818333,9.204444
27,Heumaden Bockelstraße,1,0.0,8121.534033634644,48.739722,9.227778
30,Kaltental,1,0.0,8356.404012687004,48.739722,9.130000
32,Kirchtalstraße (Volksbank Zuffenhausen),1,0.0,7182.971722788237,48.832222,9.170833
33,Kursaal,1,0.0,6929.671614009908,48.809167,9.222500


In [124]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 4, stuttgart_merged.columns[[0,1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Subway Station,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
8,Budapester Platz,2,1.0,1770.1359677851751,48.790556,9.185833
12,Daimlerplatz,6,1.0,6243.931107911371,48.805278,9.218889
50,Olgaeck,9,1.0,1006.1275402653745,48.773889,9.186111
64,Rotebühlplatz (Das Gerber),9,1.0,1142.457636958647,48.774722,9.172222
71,Schlossplatz,14,1.0,159.79046175711247,48.778889,9.178889
80,Stadtbibliothek (Handwerkskammer),1,1.0,1600.094471702996,48.790556,9.181111
95,Österreichischer Platz (WGV-Versicherungen),7,1.0,1258.8492438984558,48.770000,9.175556


In [120]:
stuttgart_merged.loc[stuttgart_merged['Cluster Labels'] == 5, stuttgart_merged.columns[[1,2,3] + list(range(5, stuttgart_merged.shape[1]))]]

,Total restaurant,Total Sushi restaurant,Distance,Subway Station Latitude,Subway Station Longitude
6,1,0.0,4521.414423314856,48.759722,9.151389
19,2,0.0,4880.416918951074,48.813611,9.168333
22,1,0.0,5042.103780061536,48.808056,9.204167
31,1,0.0,2976.5218301473747,48.799444,9.171667
37,1,0.0,4150.119417878625,48.808333,9.190000
40,2,0.0,4315.29898130593,48.811111,9.177222
41,1,0.0,5062.0902793367995,48.800000,9.211667
42,1,0.0,3269.4671502343735,48.792222,9.200556
43,2,0.0,4167.995028944893,48.796944,9.205556
44,1,0.0,2884.603650275954,48.797778,9.190278


## Results and Discussion <a name="results"></a>

Base on the cluster we can find out that cluster 0 and 4 is most intereting for us. Most of the station of Cluster 0 is near the city center stuttgart and has no Sushi restaurant within 300 meter. Cluster 4 is very near from the center and has only one Sushi restaurant nearby. 

we can also easily find out, that cluster 2, 3 and 5 are not our candidate, because they are two far from the city center and has very less restaurants nearby. They should be reasons that other don't want to open restaurant there.
For example, far from shopping area or living area. 

Cluster 1 are right in the mittle of the city and have a lot or restaurant nearby, but it has already two Sushi restaurant near each of the station. So it would be better to avoid the competions.

Finally, we can choose a platz to open the sushi restaurant base on the infomation we have. I would like to choose Schlossplatz of cluster 4, because it is at the mittle if city center that will has a really large nummer of people around for shopping or eating, even there is already one Sushi restaurant there, but the location is perfact. The other choice would be Berliner Platz, it has two station near each other with large number of restaurant nearby, and there is no Sushi restaurant there.

## Conclusion <a name="conclusion"></a>

In this project we explored and visuallized the location date of subway station and restaurant near them. we find out the total number of all restaurant and specially Sushi restautant. We also calulated the distance between subway station and city center. With all the infomation we used K means as classification method to create differeter cluster of all subway stations. Among of all these clusters we choosed two of them, which could be the candidate to open a Sushi restaurant.

In this project the location data is used as criteria for opening a restaurant. Of course, there is more criterias needed in the real world. But at least we can have some overview of the restaurants and narrow down the area we are looking for.

I hope this kind of exploration could really help the stakeholder for opening a restaurant.